## Import libraries

In [230]:
# %%capture
# !pip install catboost
# !pip install mrmr_selection
# !pip install imblearn
# !pip install mlxtend
# !pip list --format=freeze > './HSE project/Scripts/requirements.txt'

In [231]:
# Import libraries

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import seaborn as sns
from sklearn.model_selection import train_test_split
# from google.colab import output

from pandas import DatetimeIndex as dt
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
# from google.colab import files
import IPython
from IPython.display import HTML, display
# from google.colab import drive
import sys

# hyper-parameters optimisation
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# metrics
from sklearn.metrics import matthews_corrcoef as mcc
from sklearn.metrics import f1_score as f1
from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import precision_score as TP_rate                          
from sklearn.metrics import roc_auc_score as roc_auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score as recall
from sklearn.metrics import average_precision_score
from sklearn.inspection import permutation_importance
from sklearn.metrics import make_scorer,fbeta_score

# classifiers
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostRegressor, RandomForestRegressor, GradientBoostingClassifier, StackingClassifier, VotingClassifier #
from sklearn.tree import DecisionTreeClassifier     #
from sklearn.svm import SVC                                    # both linear and radial classification
from sklearn.neighbors import KNeighborsClassifier             # k=3
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
import catboost
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from mlxtend.classifier import StackingClassifier

# statistics
from scipy.stats import shapiro
from scipy.stats import chi2_contingency
from scipy.stats import mannwhitneyu

# imputations
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.base import clone
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC

# feature selection
from sklearn.feature_selection import chi2, mutual_info_classif, f_classif, SelectKBest, RFE, RFECV, SequentialFeatureSelector
from scipy.stats import kendalltau, spearmanr
from sklearn.linear_model import Lasso
from sklearn.model_selection import StratifiedKFold
from mrmr import mrmr_classif

# to conver string to dict
import ast

# Interpretability
 # !pip install interpret
from interpret.blackbox import LimeTabular
from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())
from interpret import show

import lime
import lime.lime_tabular
from __future__ import print_function

# ignore warnings when graphs are plotted
import warnings
warnings.filterwarnings('ignore')

In [232]:
# %%capture
# !pip install ipython-autotime

# %load_ext autotime

## Data editing

#### Dataset A

In [579]:
# download Dataset A from Github repo and read as excel file


link_a = 'https://github.com/KonstantinBurkin/personalized-medicine/blob/master/Data/cardio_a_updated.xlsx?raw=true'
data_a = pd.read_excel(link_a ,header=[0,1], index_col=0)
print('data_a raw shape: ', data_a.shape)

data_a raw shape:  (263, 252)


In [580]:
# Correcting data_a


# replace NAs with -1
data_a = data_a.fillna(-1)
data_a = data_a.replace(' ',-1)

# Modify 'Пол' feature: covert all string to lower format and convert to 0 and 1
data_a['АНТРОПОФИЗИОМЕТРИЯ', 'Пол'] = data_a['АНТРОПОФИЗИОМЕТРИЯ', 'Пол'].str.lower().replace(['м','ж'],[0,1])

# correct date typos
data_a['АНТРОПОФИЗИОМЕТРИЯ', 'Дата госпитализации'][154] = '2013-03-22 00:00:00'
data_a['АНТРОПОФИЗИОМЕТРИЯ', 'Дата госпитализации'][129] = '2013-01-17 00:00:00'
data_a['АНТРОПОФИЗИОМЕТРИЯ', 'Дата госпитализации'] = pd.to_datetime(data_a['АНТРОПОФИЗИОМЕТРИЯ', 'Дата госпитализации'], format='%Y-%m-%d %H:%M:%S')

# Replace typos or non-numeric data with '-1'
data_a[('СОПУТСТВУЮЩИЕ ЗАБОЛЕВАНИЯ И СОСТОЯНИЯ', 'Стадия ХБП по KDOQI /ERA')] = \
data_a[('СОПУТСТВУЮЩИЕ ЗАБОЛЕВАНИЯ И СОСТОЯНИЯ', 'Стадия ХБП по KDOQI /ERA')].replace(
    ["С2", "С1", "с2", "с1", "С3а", "с3а", "C2", "с3б", "C1", "С3б", "С4", "С3", "3А", "с4", "с5", "С3b", "ОПН!"], \
    ["c2", "c1", "c2", "c1", "c3a", "c3a", "c2", "c3b", "c1", "c3b", "c4", "c3", "c3a", "c4", "c5", "c3b", "опн!"] )
data_a[('СОПУТСТВУЮЩИЕ ЗАБОЛЕВАНИЯ И СОСТОЯНИЯ', 'Стадия ХБП по KDOQI /ERA')] = \
data_a[('СОПУТСТВУЮЩИЕ ЗАБОЛЕВАНИЯ И СОСТОЯНИЯ', 'Стадия ХБП по KDOQI /ERA')].replace(
    ['c2', 'c1', 'c3a', 'c3b', 'c4', 'c3', 0, 2, 'c5', 'опн!'], [0,  1,  2,  3,  4,  5,  6,  7,  8,  9] ).astype(int)
# ---------------
data_a[('СОПУТСТВУЮЩИЕ ЗАБОЛЕВАНИЯ И СОСТОЯНИЯ', 'Стадия по KDOQI/ERA')] = \
data_a[('СОПУТСТВУЮЩИЕ ЗАБОЛЕВАНИЯ И СОСТОЯНИЯ', 'Стадия по KDOQI/ERA')].replace(
    [-1, 'С2', 'с2', 'С1', 0, 'c2', 'с1', 'С3а', 'C2', 'C1', 'С4', 'с3б', 'С3b', 'С3А', 'с4', 'C3б', 'C3а', 'С3б'],
    [-1, 'c2', 'c2', 'c1', 0, 'c2', 'c1', 'c3a', 'c2', 'c1', 'c4', 'c3b', 'c3b', 'c3a', 'c4', 'c3b', 'c3a', 'c3b'] )
data_a[('СОПУТСТВУЮЩИЕ ЗАБОЛЕВАНИЯ И СОСТОЯНИЯ', 'Стадия по KDOQI/ERA')] = \
data_a[('СОПУТСТВУЮЩИЕ ЗАБОЛЕВАНИЯ И СОСТОЯНИЯ', 'Стадия по KDOQI/ERA')].replace(
    [-1, 'c2', 'c1', 'c3a', 'c3b', 0, 'c4'], [-1, 1, 2, 3, 4, 0, 5] ).astype(int)
# ---------------
data_a[('Хроническая сердечная недостаточность', 'НК')] = \
data_a[('Хроническая сердечная недостаточность', 'НК')].replace([-1, 1, '2а', 0, '2А', '2Б', 2, '2б'], [-1, 1, 3, 0, 3, 4, 2, 4]).astype(int)
# ---------------
data_a.rename({'Повторная реваскуляризация (ЧКВ/АКШ)': 'Повторная реваскуляризация'}, axis=1, inplace=True)
data_a[('КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ', 'Повторная реваскуляризация')].replace(to_replace=['ЧКВ',
                           'АКШ',
                           '1899-12-29 00:00:00', 
                           'ЧКВ ',
                           'АКШ ', 
                           '2018-07-30 00:00:00', 
                           '2019-04-15 00:00:00', 
                           '2020-08-30 00:00:00'], 
               value=1, 
               inplace=True)
data_a[('КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ', 'Повторная реваскуляризация')].replace(to_replace=['0'], value=0, inplace=True)

# Features with nulls: manually chose non-categorical columns with '0' and replace with '-1' 
# nulls = [col for col in data_a.columns[:90] if (len((data_a[col].unique())) > 12) and (data_a[col] == 0).sum() > 0] ; data_a[nulls]
data_a[('ЛАБОРАТОРНЫЕ ПОКАЗАТЕЛИ', 'ЛПНП, ммоль/л')] = data_a[('ЛАБОРАТОРНЫЕ ПОКАЗАТЕЛИ', 'ЛПНП, ммоль/л')].replace([0],[-1]) 
data_a[('ЛАБОРАТОРНЫЕ ПОКАЗАТЕЛИ', 'Билирубин, мкмоль/л')] = data_a[('ЛАБОРАТОРНЫЕ ПОКАЗАТЕЛИ', 'Билирубин, мкмоль/л')].replace([0],[-1]) 

In [581]:
# Find columns


hyperlipidemia = pd.DataFrame(columns=pd.MultiIndex.from_product([['ЛАБОРАТОРНЫЕ ПОКАЗАТЕЛИ'], ['Хсобщ, ммоль/л']]))
hyperlipidemia[('ЛАБОРАТОРНЫЕ ПОКАЗАТЕЛИ'), ('Хсобщ, ммоль/л')] = data_a[('ЛАБОРАТОРНЫЕ ПОКАЗАТЕЛИ', 'Хсобщ, ммоль/л')]


# drop features that were obtained at first discharge and biomarkers 
# columns must have no more than threshold=20% of NAs
threshold = 0.2
columns_with_useful_data = list(data_a.columns[:2]) + list(data_a.columns[3:77]) + list(data_a.columns[148:237])
cols_with_NAs = [col for col in columns_with_useful_data if (data_a[col] == -1).sum() > threshold*data_a.shape[0]]
data_a.drop(columns=cols_with_NAs, inplace=True)       


# find all the necessary columns
# define continuous and categorical groups
clinical_and_biomarkers = list(data_a.columns[:2]) + list(data_a.columns[3:61]) + list(data_a.columns[132:218])
continuous_cols = [col for col in clinical_and_biomarkers if (len((data_a[col].unique())) > 9)]
categorical = [col for col in clinical_and_biomarkers if (len((data_a[col].unique())) <= 9)]
clinical_and_biomarkers_and_hyperlipidemia = clinical_and_biomarkers + [('ЛАБОРАТОРНЫЕ ПОКАЗАТЕЛИ', 'Хсобщ, ммоль/л')]
# hyperlipidemia = hyperlipidemia.replace(-1, np.nan)

In [582]:
print('data_a raw shape: ', data_a.shape)

data_a raw shape:  (263, 233)


In [583]:
# For subsets


biomarkers_a_columns = list(data_a.columns[132:218])
clinical_and_biomarkers_a_columns = list(data_a.columns[:2]) + list(data_a.columns[3:61]) + list(data_a.columns[132:218])
clinical_a_columns = list(data_a.columns[:2]) + list(data_a.columns[3:61])

In [584]:
data_a[biomarkers_a_columns].head(2)

БИОМАРКЕРЫ БЛОК А                                                       \
№ п/п      TnI-1, нг/мл TnI-2, нг/мл TnI-3, нг/мл TnI-4, нг/мл CKMB-1, нг/мл   
1                  7.56         13.0        2.310        -1.00          21.5   
2                  5.17         10.3        0.304         0.66          28.2   

                                                                         ...  \
№ п/п CKMB-2, нг/мл CKMB-3, нг/мл CKMB-4, нг/мл MG-1, нг/мл MG-2, нг/мл  ...   
1              3.14          6.71          -1.0       460.6      164.20  ...   
2              0.50          3.85           7.5       421.3       56.21  ...   

                                                                              \
№ п/п АЧТВ-3, с АЧТВ-4, с АТ3-1, % АТ3-2, % АТ3-3, % АТ3-4, % FW-1,%  FW-2,%   
1          31.2      -1.0   100.72    86.59   116.31    -1.00  120.0  120.00   
2          34.0      36.1   100.55   101.63   127.76    99.58  120.0  115.99   

                     
№ п/п FW-3,% FW-4,%  
1        293   -1.0  
2         90   94.0  

[2 rows x 86 columns]

In [585]:
data_a[clinical_and_biomarkers_a_columns].head(2)

АНТРОПОФИЗИОМЕТРИЯ                                                       \
№ п/п                Пол Возраст  Рост  Вес        ИМТ S тела систол. АД  ЧСС   
1                      0      75  1.64   80  29.744200   1.88        190   90   
2                      0      49  1.76  130  41.967975   2.41        140  100   

              ХАРАКТЕРИСТИКА ОИМ             ... БИОМАРКЕРЫ БЛОК А            \
№ п/п Давность болевого синдрома Cегмент ST  ...         АЧТВ-3, с АЧТВ-4, с   
1                              2          1  ...              31.2      -1.0   
2                              2          1  ...              34.0      36.1   

                                                                        
№ п/п АТ3-1, % АТ3-2, % АТ3-3, % АТ3-4, % FW-1,%  FW-2,% FW-3,% FW-4,%  
1       100.72    86.59   116.31    -1.00  120.0  120.00    293   -1.0  
2       100.55   101.63   127.76    99.58  120.0  115.99     90   94.0  

[2 rows x 146 columns]

In [586]:
data_a[clinical_a_columns].head(2)

АНТРОПОФИЗИОМЕТРИЯ                                                       \
№ п/п                Пол Возраст  Рост  Вес        ИМТ S тела систол. АД  ЧСС   
1                      0      75  1.64   80  29.744200   1.88        190   90   
2                      0      49  1.76  130  41.967975   2.41        140  100   

              ХАРАКТЕРИСТИКА ОИМ             ... ИСХОДНАЯ ЭХОКГ          \
№ п/п Давность болевого синдрома Cегмент ST  ...          ФВ ЛЖ ТМЖП ЛЖ   
1                              2          1  ...      55.147059     1.3   
2                              2          1  ...      54.304636     1.4   

                                                                     
№ п/п ТЗС ЛЖ  ММ ЛЖ иММ ЛЖ   ЛП РМК Аневризма ЛЖ Тромбоз ЛЖ ИНЛС ЛЖ  
1       1.20  271.5  144.4  4.2   1            0          0   0.000  
2       1.13  301.6  125.1  4.1   1            0          0   1.125  

[2 rows x 60 columns]

#### Dataset B

In [587]:
# download Dataset A from Github repo and read as excel file


link_b = 'https://github.com/KonstantinBurkin/personalized-medicine/blob/master/Data/cardio_b_updated.xlsx?raw=true'
data_b = pd.read_excel(link_b ,header=[0,1], index_col=0)
print('data_b raw shape: ', data_b.shape)

data_b raw shape:  (109, 232)


In [588]:
# Correcting data_b


# replace NAs with -1
data_b = data_b.fillna(-1)
data_b = data_b.replace(' ',-1)
# Modify 'Пол' feature: covert all string to lower format and convert to 0 and 1
data_b['АНТРОПОФИЗИОМЕТРИЯ', 'Пол'] = data_b['АНТРОПОФИЗИОМЕТРИЯ', 'Пол'].str.lower().replace(['м','ж'],[0,1])

# Replace typos or non-numeric data with '-1'
data_b[('ИСХОДНАЯ ЭХОКГ', 'РМК')] = data_b[('ИСХОДНАЯ ЭХОКГ', 'РМК')].replace('1-2', -1).astype(int)
# ---------------
data_b[('КОРОНАРОАНГИОГРАФИЯ И РЕВАСКУЛЯРИЗАЦИЯ МИОКАРДА', 'TIMI в ИЗА')] = \
data_b[('КОРОНАРОАНГИОГРАФИЯ И РЕВАСКУЛЯРИЗАЦИЯ МИОКАРДА', 'TIMI в ИЗА')].replace(['0-I', '0-1', '1-2'], [4, 5, 6]).astype(int)
# ---------------
data_b[('СОПУТСТВУЮЩИЕ ЗАБОЛЕВАНИЯ И СОСТОЯНИЯ', 'Стадия ХБП по KDOQI /ERA')] = \
data_b[('СОПУТСТВУЮЩИЕ ЗАБОЛЕВАНИЯ И СОСТОЯНИЯ', 'Стадия ХБП по KDOQI /ERA')].replace(
    ["С2", "С1", "с2", "с1", "С3а", "с3а", "C2", "с3б", "C1", "С3б", "С4", "С3", "3А", "с4", "с5", "С3b", "ОПН!"], \
    ["c2", "c1", "c2", "c1", "c3a", "c3a", "c2", "c3b", "c1", "c3b", "c4", "c3", "3a", "c4", "c5", "c3b", "опн!"] )
data_b[('СОПУТСТВУЮЩИЕ ЗАБОЛЕВАНИЯ И СОСТОЯНИЯ', 'Стадия ХБП по KDOQI /ERA')] = \
data_b[('СОПУТСТВУЮЩИЕ ЗАБОЛЕВАНИЯ И СОСТОЯНИЯ', 'Стадия ХБП по KDOQI /ERA')].replace(
    [0, 'c1', 2, 'c2', 'c3', '3a', 'c3a', 'c3b', 'c4', 'c5', 'опн!'], [0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10] ).astype(int)
# ---------------
data_b[('ЭХОКГ (ИСХОД)', 'ТМЖП ЛЖ')] = \
data_b[('ЭХОКГ (ИСХОД)', 'ТМЖП ЛЖ')].replace(['1,1-1,0-0,8', '1,0 - 1,5', '1,3; 0,8'], -1).astype(float)
# ---------------
data_b[('ПОВТОРНАЯ ЭХОКГ', 'ЛП')] = \
data_b[('ПОВТОРНАЯ ЭХОКГ', 'ЛП')].replace(['С2'], -1).astype(float)
# ---------------
data_b[('ПОВТОРНАЯ ЭХОКГ', 'ТМЖП ЛЖ')] = \
data_b[('ПОВТОРНАЯ ЭХОКГ', 'ТМЖП ЛЖ')].replace(['1,1-1.0-0,7'], -1).astype(float)
# ---------------
data_b.replace(to_replace=['ЧКВ',
                           'АКШ',
                           '1899-12-29 00:00:00', 
                           'ЧКВ ',
                           'АКШ ', 
                           pd.to_datetime('2018-07-30 00:00:00', format='%Y-%m-%d %H:%M:%S'),
                           pd.to_datetime('2019-04-15 00:00:00', format='%Y-%m-%d %H:%M:%S'),
                           pd.to_datetime('2020-08-30 00:00:00', format='%Y-%m-%d %H:%M:%S'),
                           ], 
               value=1, 
               inplace=True)
data_b[('КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ', 'Повторная реваскуляризация')].replace(to_replace=['0'], value=0, inplace=True)

# Features with nulls: manually chose non-categorical columns with '0' and replace with '-1' 
# nulls = [col for col in data_b.columns[:90] if (len((data_b[col].unique())) > 12) and (data_b[col] == 0).sum() > 0] ; data_b[nulls]
data_b[('ИСХОДНАЯ ЭХОКГ', 'ИНЛС ЛЖ')] = data_b[('ИСХОДНАЯ ЭХОКГ', 'ИНЛС ЛЖ')].replace([0],[-1]) 

In [589]:
# Find columns


# drop features that were obtained at first discharge and biomarkers that have more than 20% of NAs
cols_with_NAs = [col for col in (list(data_b.columns[:2]) + list(data_b.columns[3:84])) \
                        if (data_b[col] == -1).sum() > 0.2*data_b.shape[0]] # there are more than 20% of NAs in some biomarkers
data_b.drop(columns=cols_with_NAs, inplace=True)

      


clinical_and_biomarkers = list(data_b.columns[:2]) + list(data_b.columns[3:72]) + list(data_b.columns[214:])
continuous_cols = [col for col in clinical_and_biomarkers if (len((data_b[col].unique())) >= 7)]
categorical = [col for col in clinical_and_biomarkers if (len((data_b[col].unique())) < 7)]


In [590]:
# For subsets
biomarkers_b_columns = list(data_b.columns[214:])
clinical_and_biomarkers_b_columns = list(data_b.columns[:2]) + list(data_b.columns[3:72]) + list(data_b.columns[214:])
clinical_b_columns = list(data_b.columns[:2]) + list(data_b.columns[3:72])

In [591]:
data_b[biomarkers_b_columns].head(2)

БИОМАРКЕРЫ БЛОК Е                                                     \
№ п/п            БСЖК-1 БСЖК-2 hsТnT-2, пг/мл hsТnT-2, пг/мл.1 MG-1, нг/мл   
1                     1      1           9.52           1200.0       24.00   
2                     0      0          19.52           1175.0       62.83   

                   
№ п/п MG-2, нг/мл  
1           62.72  
2           36.88

In [592]:
data_b[clinical_and_biomarkers_b_columns].head(2)

АНТРОПОФИЗИОМЕТРИЯ                                                      \
№ п/п                Пол Возраст  Рост  Вес        ИМТ S тела систол. АД ЧСС   
1                      1      68  1.60   52  20.312500   1.51        140  65   
2                      0      50  1.84  115  33.967391   2.38        134  67   

      ПСИХОСОЦИАЛЬНЫЕ ФАКТОРЫ         ХАРАКТЕРИСТИКА ОИМ  ... ИСХОДНАЯ ЭХОКГ  \
№ п/п                 Курение Давность болевого синдрома  ...     Тромбоз ЛЖ   
1                           0                          1  ...              0   
2                           1                          1  ...              0   

                КЛИРЕНС 2       БИОМАРКЕРЫ БЛОК Е                        \
№ п/п ИНЛС ЛЖ Креатинин 2   СКФ            БСЖК-1 БСЖК-2 hsТnT-2, пг/мл   
1      1.0625        58.0  95.0                 1      1           9.52   
2      1.0625       126.0  56.0                 0      0          19.52   

                                                
№ п/п hsТnT-2, пг/мл.1 MG-1, нг/мл MG-2, нг/мл  
1               1200.0       24.00       62.72  
2               1175.0       62.83       36.88  

[2 rows x 77 columns]

In [593]:
data_b[clinical_b_columns].head(2)

АНТРОПОФИЗИОМЕТРИЯ                                                      \
№ п/п                Пол Возраст  Рост  Вес        ИМТ S тела систол. АД ЧСС   
1                      1      68  1.60   52  20.312500   1.51        140  65   
2                      0      50  1.84  115  33.967391   2.38        134  67   

      ПСИХОСОЦИАЛЬНЫЕ ФАКТОРЫ         ХАРАКТЕРИСТИКА ОИМ  ... ИСХОДНАЯ ЭХОКГ  \
№ п/п                 Курение Давность болевого синдрома  ...         иММ ЛЖ   
1                           0                          1  ...          137.5   
2                           1                          1  ...          104.0   

                                                           КЛИРЕНС 2        
№ п/п   ЛП   ПП   ПЖ РМК Аневризма ЛЖ Тромбоз ЛЖ ИНЛС ЛЖ Креатинин 2   СКФ  
1      2.8  2.7  2.6  -1            0          0  1.0625        58.0  95.0  
2      3.7  3.7  3.0   1            0          0  1.0625       126.0  56.0  

[2 rows x 71 columns]

#### Dataset C

In [594]:
# download Dataset A from Github repo and read as excel file


link_c = 'https://github.com/KonstantinBurkin/personalized-medicine/blob/master/Data/%D0%91%D0%BB%D0%BE%D0%BA%20%D0%A1%C2%A0%D0%BE%D0%B1%D0%B5%D0%B7%D0%BB%D0%B8%D1%87.xlsx?raw=true'
data_c = pd.read_excel(link_c ,header=[0,1], index_col=0)
print('data_c raw shape: ', data_c.shape)

data_c raw shape:  (129, 470)


In [595]:
# Correcting data_c


# replace NAs with -1
data_c = data_c.fillna(-1)
data_c = data_c.replace(' ',-1)
# data_c.columns[(data_c.dtypes == object).values]
# Modify 'Пол' feature: covert all string to lower format and convert to 0 and 1
data_c['АНТРОПОФИЗИОМЕТРИЯ', 'Пол'] = data_c['АНТРОПОФИЗИОМЕТРИЯ', 'Пол'].replace([1, 2],[0,1])
# Reencrypt psychological scales
data_c['ПСИХОСОЦИАЛЬНЫЕ ФАКТОРЫ', 'Ситуативная тревожность Спилберга (баллы)'].replace(range(31), 0, inplace=True)
data_c['ПСИХОСОЦИАЛЬНЫЕ ФАКТОРЫ', 'Ситуативная тревожность Спилберга (баллы)'].replace(range(31, 46), 1, inplace=True)
data_c['ПСИХОСОЦИАЛЬНЫЕ ФАКТОРЫ', 'Ситуативная тревожность Спилберга (баллы)'].replace(range(46,100), 2, inplace=True)
data_c['ПСИХОСОЦИАЛЬНЫЕ ФАКТОРЫ', 'Личная тревожность Спилберга (баллы)'].replace(range(31), 0, inplace=True)
data_c['ПСИХОСОЦИАЛЬНЫЕ ФАКТОРЫ', 'Личная тревожность Спилберга (баллы)'].replace(range(31, 46), 1, inplace=True)
data_c['ПСИХОСОЦИАЛЬНЫЕ ФАКТОРЫ', 'Личная тревожность Спилберга (баллы)'].replace(range(46,100), 2, inplace=True)
data_c['ПСИХОСОЦИАЛЬНЫЕ ФАКТОРЫ', 'Опросник депрессии Бека (баллы)'].replace(range(10), 0, inplace=True)
data_c['ПСИХОСОЦИАЛЬНЫЕ ФАКТОРЫ', 'Опросник депрессии Бека (баллы)'].replace(range(10, 20), 1, inplace=True)
data_c['ПСИХОСОЦИАЛЬНЫЕ ФАКТОРЫ', 'Опросник депрессии Бека (баллы)'].replace(range(20,100), 2, inplace=True)
data_c['ПСИХОСОЦИАЛЬНЫЕ ФАКТОРЫ', 'Шкала AUDIT (баллы)'].replace(range(16), 0, inplace=True)
data_c['ПСИХОСОЦИАЛЬНЫЕ ФАКТОРЫ', 'Шкала AUDIT (баллы)'].replace(range(16, 100), 1, inplace=True)

date_c = pd.to_datetime(data_c['АНТРОПОФИЗИОМЕТРИЯ', 'Дата госпитализации'], format='%Y-%m-%d %H:%M:%S')
data_c.drop(columns=[('АНТРОПОФИЗИОМЕТРИЯ', 'Дата рождения'), ('АНТРОПОФИЗИОМЕТРИЯ', 'Дата госпитализации')], inplace=True)
data_c.drop(columns=['ПЛАНОВАЯ ТЕРАПИЯ ПРИ ВЫПИСКЕ', 'ГОСПИТАЛЬНАЯ ТЕРАПИЯ'], inplace=True)


In [596]:
# Find columns


# cols with features from first discharge and biomarkers
cols_of_interest = list(data_c.columns[:153]) + list(data_c.columns[419:424])

# drop features that were obtained at first discharge and biomarkers that have more than 20% of NAs
cols_with_NAs = [col for col in cols_of_interest[:-5] if (data_c[col] == -1).sum() > 0.2*data_c.shape[0]]
cols_of_interest = [col for col in cols_of_interest if col not in cols_with_NAs]
continuous_cols = [col for col in cols_of_interest if (len((data_c[col].unique())) > 9)]
categorical = [col for col in cols_of_interest if (len((data_c[col].unique())) <= 9)]

In [597]:
# For subsets


biomarkers_c_columns = cols_of_interest[-5:]
clinical_c_columns = cols_of_interest[:-5]
clinical_and_biomarkers_c_columns = clinical_c_columns + biomarkers_c_columns

#### Сombined Dataset ABC

In [598]:
# merge clinicals of all three datasets
data_abc = pd.concat([data_a, data_b, data_c], axis=0)
data_abc.dropna(axis=1, inplace=True)

cols_of_interest = list(data_abc.columns)[:56]

In [599]:
# For subsets
clinical_abc_columns = cols_of_interest

In [600]:
data_abc[clinical_abc_columns].head()

АНТРОПОФИЗИОМЕТРИЯ                                                       \
№ п/п                Пол Возраст  Рост  Вес        ИМТ S тела систол. АД  ЧСС   
1                      0      75  1.64   80  29.744200   1.88        190   90   
2                      0      49  1.76  130  41.967975   2.41        140  100   
3                      0      54  1.62   70  26.672763   1.76        137   86   
4                      0      33  1.76   90  29.054752   2.07        128   76   
5                      0      52  1.68   90  31.887755   2.02        160  105   

              ХАРАКТЕРИСТИКА ОИМ             ... ИСХОДНАЯ ЭХОКГ          \
№ п/п Давность болевого синдрома Cегмент ST  ...          ФВ ЛЖ ТМЖП ЛЖ   
1                              2          1  ...      55.147059    1.30   
2                              2          1  ...      54.304636    1.40   
3                              1          0  ...      69.047619    1.20   
4                              1          2  ...      45.384615    1.09   
5                              3          1  ...      55.279503    1.60   

                                                                       
№ п/п ТЗС ЛЖ   ММ ЛЖ иММ ЛЖ   ЛП  РМК Аневризма ЛЖ Тромбоз ЛЖ ИНЛС ЛЖ  
1       1.20  271.50  144.4  4.2  1.0          0.0        0.0   0.000  
2       1.13  301.60  125.1  4.1  1.0          0.0        0.0   1.125  
3       1.10  234.60  133.3  3.8  0.0          0.0        0.0   0.000  
4       1.10  219.39  106.0  3.8  1.0          0.0        0.0   1.125  
5       0.80  288.67  142.9  4.7  2.0          0.0        0.0   1.125  

[5 rows x 56 columns]

## Functions

In [620]:
def get_combined_target_column(dataset):
    '''This function gets combined target values.'''

    target_combined = (((dataset['КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ']['Сердечно-сосудистая смерть'].astype(int) == 1)|
                        (dataset['КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ']['Реинфаркт'].astype(int) == 1) |
                        (dataset['КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ']['ОНМК'].astype(int) == 1)|
                        (dataset['КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ']['Повторная реваскуляризация'].astype(int) == 1))&
                        (dataset['ГОСПИТАЛЬНЫЕ КЛИНИЧЕСКИЕ ИСХОДЫ']['Смерть']!=1))*1

    mask_combined = ((dataset['КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ']['Сердечно-сосудистая смерть'].astype(int) == -1) &
                    (dataset['КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ']['Реинфаркт'].astype(int) == -1) &
                    (dataset['КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ']['ОНМК'].astype(int) == -1) &
                    (dataset['КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ']['Повторная реваскуляризация'].astype(int) == -1))     
    target_combined[mask_combined] = -1
    return target_combined 

In [632]:
def imputer(train, test, cat_features_id):
    '''This function imputes missing values with IterativeImputer.
       RandomForest is used as an estimator.'''

    # Impute NAs with IterativeImputer (estimator - RandomForestRegressor)
    impute_estimator = RandomForestRegressor(n_estimators=50,
                                            max_depth=5,
                                            n_jobs=-1,
                                            random_state=random_state)
    imputer = IterativeImputer(random_state=random_state,
                              estimator=impute_estimator, 
                              max_iter=25)
    train = imputer.fit_transform(train)
    test = imputer.transform(test)

    # Round values for categorical data - so that there will be no new categories
    train[:, categorical_features_index] = train[:, categorical_features_index].round()
    test[:, categorical_features_index] = test[:, categorical_features_index].round()

    return train, test

In [630]:
def scaler(train, test, continuous_features):
    '''This function scales continuous features
       with Robust Scaler.'''

    scaler = RobustScaler() 
    train[continuous_cols] = scaler.fit_transform(train[continuous_cols])
    test[continuous_cols] = scaler.transform(test[continuous_cols])

    return train, test

In [628]:
def imputation_smote(random_state, dataset, target, path, dataset_features, test_size, name, download=False):
    '''This function is pipeline for dataset preprocessing:
       Finds subset with features that we will use further.
       Adds target column.
       Drops patients with target == -1.
       Splits subset into train and test.
       Imputes subsets with Iterative Imputer.
       Over-samples Train subset.
       Downloads train and test datasets.'''

    if target == ('target', 'combined'):
        dataset[target] = get_combined_target_column(dataset)

    # add target columnt and drop patients with -1 in the outcome
    dataframe = dataset[dataset_features + [target]].copy()
    dataframe[target] = dataframe[target].replace(-1, np.nan)
    dataframe.dropna(axis=0, how='any', inplace=True)
    dataframe.replace(-1, np.nan, inplace=True)

    # divide dataset into train and test
    X_train, X_test, y_train, y_test = \
    train_test_split(dataframe[dataset_features], 
                    dataframe[target], 
                    test_size=test_size, 
                    random_state=random_state, 
                    # stratify=dataframe[target],
                    shuffle=True)

    # Impute NAs with IterativeImputer (estimator - RandomForestRegressor)
    categorical_features_index = [dataframe.columns.get_loc(col) for col in dataframe.columns[:-1] if len(dataframe[col].unique()) <= 9]
    X_train, X_test = imputer(train=X_train, test=X_test, cat_features_id=categorical_features_index)

    # fit SMOTE on train part
    smote = SMOTENC(categorical_features = categorical_features_index,
                    sampling_strategy='minority',
                    n_jobs=-1,
                    random_state=random_state)
    X_sm, y_sm = smote.fit_resample(X_train, y_train)

    train_imputed = pd.DataFrame(data=X_sm, columns=dataset_features)
    train_imputed[target] = y_sm
    test_imputed = pd.DataFrame(data=X_test, columns=dataset_features)
    test_imputed[target] = y_test.values

    continuous_cols = [col for col in dataset_features if (len((train_imputed[col].unique())) > 9)]

    # Scaling of dataset
    train_imputed, test_imputed = scaler(train=train_imputed, test=test_imputed, continuous_features=continuous_cols)

    print('Train shape:\t', train_imputed.shape)
    print('Train target:\n', train_imputed[target].value_counts(), end='')
    print('Test shape:\t', test_imputed.shape)
    print('Test target:\n', test_imputed[target].value_counts(), end='')

    # download dataset
    if download:
        train_imputed.to_excel(f'{path}train_{name}.xlsx')
        test_imputed.to_excel(f'{path}test_{name}.xlsx')
    
    return None

## Lancet Dataset ABC

In [457]:
data_a_and_hyperlipidemia = data_a.copy()
data_a_and_hyperlipidemia[('ЛАБОРАТОРНЫЕ ПОКАЗАТЕЛИ'), ('Хсобщ, ммоль/л')] = hyperlipidemia

data_abc_lancet = pd.concat([data_a_and_hyperlipidemia, data_b, data_c], axis=0)
data_abc_lancet.dropna(axis=1, inplace=True)

# cols_of_interest = list(data_abc_lancet.columns)[:56]
# cols_of_interest.append(('ЛАБОРАТОРНЫЕ ПОКАЗАТЕЛИ', 'Хсобщ, ммоль/л'))

data_abc_lancet[('target', 'Cмерть')] = data_abc_lancet[[('КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ', 'Некардиальная смерть'),
          ('КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ', 'Сердечно-сосудистая смерть')]].apply((lambda x: x.max()), axis=1)

In [458]:
# For subsets
clinical_abc_columns_lancet = [
    # age
    ('АНТРОПОФИЗИОМЕТРИЯ', 'Возраст'),
    # sex
    ('АНТРОПОФИЗИОМЕТРИЯ', 'Пол'),
    # diabetes
    ('СОПУТСТВУЮЩИЕ ЗАБОЛЕВАНИЯ И СОСТОЯНИЯ', 'СД'),
    # hypertension
    ('СОПУТСТВУЮЩИЕ ЗАБОЛЕВАНИЯ И СОСТОЯНИЯ', 'ГБ'),
    # hyperlipidaemia
    ('ЛАБОРАТОРНЫЕ ПОКАЗАТЕЛИ', 'Хсобщ, ммоль/л'), # !!!!!!!!!!!!!!!!!!  this feature was dropped in data_a можно сделать отдельный датасет для lancet
    # peripheral artery disease
    ('СОПУТСТВУЮЩИЕ ЗАБОЛЕВАНИЯ И СОСТОЯНИЯ', 'МФА'),
    # EGFR
    ('ЛАБОРАТОРНЫЕ ПОКАЗАТЕЛИ', 'СКФ EPI'),
    # previous myocardial infarction
    ('СОПУТСТВУЮЩИЕ ЗАБОЛЕВАНИЯ И СОСТОЯНИЯ', 'пост-ИМ'),
    # previous percutaneous coronary intervention
    ('СОПУТСТВУЮЩИЕ ЗАБОЛЕВАНИЯ И СОСТОЯНИЯ', 'пост-стент'),
    # previous stroke
    ('СОПУТСТВУЮЩИЕ ЗАБОЛЕВАНИЯ И СОСТОЯНИЯ', 'пост-ОНМК'),
    # previous bleeding
    ('СОПУТСТВУЮЩИЕ ЗАБОЛЕВАНИЯ И СОСТОЯНИЯ', 'пост-ВЧ-кровоизлияние'),
    # ST-segment elevation myocardial infarction [STEMI] presentation
    ('ХАРАКТЕРИСТИКА ОИМ', 'Cегмент ST'),
    # haemoglobin
    ('ЛАБОРАТОРНЫЕ ПОКАЗАТЕЛИ', 'Гемоглобин, г/л'),
    # left ventricular ejection fraction [LVEF])
    ('ИСХОДНАЯ ЭХОКГ', 'ФВ ЛЖ'),
    # target
    # ('target', 'Cмерть')
]

In [460]:
imputation_smote(
random_state = 20,
dataset = data_abc_lancet,
target = ('target', 'Cмерть'),
path = "./HSE project/Preprocessed Data/lancet dataset/",
dataset_features = clinical_abc_columns_lancet,
test_size = 0.25, 
download=True,
name='abc_lancet'
)

Train shape:	 (450, 15)
Train target:
 0.0    225
1.0    225
Name: (target, Cмерть), dtype: int64
Test shape:	 (105, 15)
Test target:
 0.0    76
1.0    29
Name: (target, Cмерть), dtype: int64


## **Target**: Cardiovascular death

#### Dataset A

In [387]:
imputation_smote(
random_state = 20,
dataset = data_a,
target = ('КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ', 'Сердечно-сосудистая смерть'),
path = "./HSE project/Preprocessed Data/cardiovascular death/",
dataset_features = clinical_and_biomarkers_a_columns,
test_size = 0.25, 
name='a',
download=True)

Train shape:	 (150, 147)
Train target:
 0.0    75
1.0    75
Name: (КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ, Сердечно-сосудистая смерть), dtype: int64
Test shape:	 (49, 147)
Test target:
 0.0    29
1.0    20
Name: (КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ, Сердечно-сосудистая смерть), dtype: int64


#### Dataset B

In [388]:
imputation_smote(
random_state = 20,
dataset = data_b,
target = ('КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ', 'Сердечно-сосудистая смерть'),
path = "./HSE project/Preprocessed Data/cardiovascular death/",
dataset_features = clinical_and_biomarkers_b_columns,
test_size = 0.33, 
name = 'b',
download=True
)

Train shape:	 (94, 78)
Train target:
 0.0    47
1.0    47
Name: (КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ, Сердечно-сосудистая смерть), dtype: int64
Test shape:	 (30, 78)
Test target:
 0.0    24
1.0     6
Name: (КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ, Сердечно-сосудистая смерть), dtype: int64


#### Dataset C

In [413]:
imputation_smote(
random_state = 20,
dataset = data_c,
target = ('КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ', 'Сердечно-сосудистая смерть'),
path = "./HSE project/Preprocessed Data/cardiovascular death/",
dataset_features = clinical_and_biomarkers_c_columns,
test_size = 0.33, 
download=True,
name='c'
)

Train shape:	 (160, 106)
Train target:
 0    80
1    80
Name: (КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ, Сердечно-сосудистая смерть), dtype: int64
Test shape:	 (43, 106)
Test target:
 0    42
1     1
Name: (КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ, Сердечно-сосудистая смерть), dtype: int64


#### Сombined Dataset ABC

In [420]:
imputation_smote(
random_state = 20,
dataset = data_abc,
target = ('КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ', 'Сердечно-сосудистая смерть'),
path = "./HSE project/Preprocessed Data/cardiovascular death/",
dataset_features = clinical_abc_columns,
test_size = 0.25, 
download=True,
name='abc'
)

Train shape:	 (438, 57)
Train target:
 0.0    219
1.0    219
Name: (КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ, Сердечно-сосудистая смерть), dtype: int64
Test shape:	 (104, 57)
Test target:
 0.0    78
1.0    26
Name: (КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ, Сердечно-сосудистая смерть), dtype: int64


## **Target**: Revascularization

#### Dataset A

In [461]:
imputation_smote(
random_state = 20,
dataset = data_a,
target = ('КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ', 'Повторная реваскуляризация'),
path = "./HSE project/Preprocessed Data/revascularization/",
dataset_features = clinical_and_biomarkers_a_columns,
test_size = 0.25, 
name='a',
download=True)

Train shape:	 (122, 147)
Train target:
 0.0    61
1.0    61
Name: (КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ, Повторная реваскуляризация), dtype: int64
Test shape:	 (33, 147)
Test target:
 0.0    23
1.0    10
Name: (КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ, Повторная реваскуляризация), dtype: int64


#### Dataset B

In [462]:
imputation_smote(
random_state = 20,
dataset = data_b,
target = ('КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ', 'Повторная реваскуляризация'),
path = "./HSE project/Preprocessed Data/revascularization/",
dataset_features = clinical_and_biomarkers_b_columns,
test_size = 0.33, 
name = 'b',
download=True
)

Train shape:	 (86, 78)
Train target:
 1.0    43
0.0    43
Name: (КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ, Повторная реваскуляризация), dtype: int64
Test shape:	 (29, 78)
Test target:
 0.0    24
1.0     5
Name: (КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ, Повторная реваскуляризация), dtype: int64


#### Dataset C

In [463]:
imputation_smote(
random_state = 20,
dataset = data_c,
target = ('КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ', 'Повторная реваскуляризация'),
path = "./HSE project/Preprocessed Data/revascularization/",
dataset_features = clinical_and_biomarkers_c_columns,
test_size = 0.33, 
download=True,
name='c'
)

Train shape:	 (146, 106)
Train target:
 1    73
0    73
Name: (КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ, Повторная реваскуляризация), dtype: int64
Test shape:	 (43, 106)
Test target:
 0    38
1     5
Name: (КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ, Повторная реваскуляризация), dtype: int64


#### Сombined Dataset ABC

In [464]:
imputation_smote(
random_state = 20,
dataset = data_abc,
target = ('КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ', 'Повторная реваскуляризация'),
path = "./HSE project/Preprocessed Data/revascularization/",
dataset_features = clinical_abc_columns,
test_size = 0.25, 
download=True,
name='abc'
)

Train shape:	 (390, 57)
Train target:
 0.0    195
1.0    195
Name: (КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ, Повторная реваскуляризация), dtype: int64
Test shape:	 (87, 57)
Test target:
 0.0    67
1.0    20
Name: (КОНЕЧНЫЕ ИСХОДЫ НАБЛЮДЕНИЯ, Повторная реваскуляризация), dtype: int64


## **Target**: Combined

#### Dataset A

In [562]:
imputation_smote(
random_state = 20,
dataset = data_a,
target = ('target', 'combined'),
path = "./HSE project/Preprocessed Data/combined/",
dataset_features = clinical_and_biomarkers_a_columns,
test_size = 0.25, 
name='a',
download=True)

Train shape:	 (192, 147)
Train target:
 0.0    96
1.0    96
Name: (target, combined), dtype: int64
Test shape:	 (50, 147)
Test target:
 1.0    27
0.0    23
Name: (target, combined), dtype: int64


#### Dataset B

In [565]:
imputation_smote(
random_state = 20,
dataset = data_b,
target = ('target', 'combined'),
path = "./HSE project/Preprocessed Data/combined/",
dataset_features = clinical_and_biomarkers_b_columns,
test_size = 0.33, 
name = 'b',
download=True
)

Train shape:	 (86, 78)
Train target:
 1.0    43
0.0    43
Name: (target, combined), dtype: int64
Test shape:	 (32, 78)
Test target:
 0.0    23
1.0     9
Name: (target, combined), dtype: int64


#### Dataset C

In [631]:
imputation_smote(
random_state = 20,
dataset = data_c,
target = ('target', 'combined'),
path = "./HSE project/Preprocessed Data/combined/",
dataset_features = clinical_and_biomarkers_c_columns,
test_size = 0.33, 
# download=True,
name='c'
)

Train shape:	 (120, 106)
Train target:
 1    60
0    60
Name: (target, combined), dtype: int64Test shape:	 (43, 106)
Test target:
 0    34
1     9
Name: (target, combined), dtype: int64

#### Сombined Dataset ABC

In [567]:
imputation_smote(
random_state = 20,
dataset = data_abc,
target = ('target', 'combined'),
path = "./HSE project/Preprocessed Data/combined/",
dataset_features = clinical_abc_columns,
test_size = 0.25, 
download=True,
name='abc'
)

Train shape:	 (358, 57)
Train target:
 0.0    179
1.0    179
Name: (target, combined), dtype: int64
Test shape:	 (106, 57)
Test target:
 0.0    58
1.0    48
Name: (target, combined), dtype: int64


## References and info

Letter with data: [here](https://mail.yandex.ru/?win=176&clid=1985545-207&uid=112725799#message/179862510118127643)
- [Difference between Standard scaler and MinMaxScaler](https://stackoverflow.com/questions/51237635/difference-between-standard-scaler-and-minmaxscaler)
- [sklearn: Compare the effect of different scalers on data with outliers](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html)
- [How to Use StandardScaler and MinMaxScaler Transforms in Python](https://machinelearningmastery.com/standardscaler-and-minmaxscaler-transforms-in-python/)

Robust scaler is less sensitive to outliers -> might perform better  
Although, for non-normal distribution of variables Normalisation methods (MinMaxScaler) are recommended